In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
import time
from pathlib import Path
from datetime import datetime, date, timedelta

In [ ]:
# 設定縣市代碼與對應的縣市名稱
city_dict = {
    "A":"NewTaipei",
    "V":"Taipei",
    "S":"Taichung",
    "U":"Tainan",
    "W":"Kaohsiung",
    "C":"Taoyuan",
    "B":"Yilan",
    "D":"Hsinchu",
    "E":"Miaoli",
    "G":"Changhua",
    "H":"Nantou",
    "I":"Yunlin",
    "J":"Chiayi",
    "M":"Pingtung",
    "N":"Taitung",
    "O":"Hualien",
    "P":"Penghu",
    "Q":"Keelung",
    "R":"HsinchuCity",
    "T":"ChiayiCity",
    "Y":"Kinmen",
    "X":"Lianjiang"
}

# 將代碼另存為list，方便迴圈
city_list = list(city_dict.keys())


# 根據縣市代碼進行迴圈
def main():
    for city in city_list:
        print(f"開始查詢{city_dict[city]}的資料...")

        # 每次都建立新的DF
        columns = [
            'BussName',
            'PBLicense',
            'BiName',
            'AnimalType',
            'EvaluationLevelName',
            'IsActive',
            'EffectiveDate',
            'Principal',
            'Staff',
            'CountyName',
            'CTownName',
            'CAddress',
            'ActiveSDate',
            'EvaluationDataShow',
            'EvaluationComment',
            'EvaluationCommentShow',
            'RowNum',
            'QueryRowsCount']

        main_df = pd.DataFrame(columns=columns)


        # 設定網址、headers、起始頁數
        url = "https://www.pet.gov.tw/handler/PetBusinessCore.ashx"
        headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36"}
        page = 1

        # 開始迴圈，每次迴圈頁數+1，直到回傳結果出現"查無資料"才會停止迴圈
        while True:
            data = {
                "Method":"DailyWorkBE",
                "Param":json.dumps({
                    "action":"GetBusinessList2",
                    "_BusinessList":{
                        "Countycode":city,  # 縣市
                        "PBLicense":"",
                        "BI":"",
                        "BussName":"",
                        "EvaluationLevel":"",
                        "KeyWord":"",
                        "pageSize":100,    # 每頁顯示筆數
                        "currentPage":page, # 頁數
                        "IsActive":"1"}
                })
            }

            max_tries = 3
            for tries in range(1, max_tries+1):
                try:
                    res = requests.post(url=url, headers=headers, data=data)
                    res.encoding = "utf-8-sig"
                    break

                except Exception as e:
                    if tries >= max_tries:
                        print("已達到最大嘗試次數，跳過該日")
                        break
                    else:
                        print(f"第{tries}次失敗，錯誤：{e}，{tries*5}秒後重試...")
                        time.sleep(tries*5)
                        continue

            if res.json()["ErrorMessage"] == "查無資料":
                break
            else:
                data_ori = res.json()
                data = data_ori.get("Message", [])
                data_json = json.loads(data)
                df = pd.DataFrame(data_json)

                main_df = pd.concat([main_df, df])

                page += 1
                time.sleep(5)


        # 取得今天日期，作為更新日
        today = datetime.now().date()
        main_df["update_date"] = today

        # 如有名字有重複則去除，並重設index
        main_df = main_df.drop_duplicates(subset="BussName", keep="last")
        main_df = main_df.reset_index(drop=True)

        # 直接覆蓋掉原本的檔案（假設每次查詢皆為最新狀態）
        file_path = f"C:/Users/add41/Documents/Data_Engineer/Project/example_data/pet_buss/{city_dict[city]}_bussiness.csv"
        main_df.to_csv(file_path, index=False)
        print(f"已更新{city_dict[city]}的資料")



if __name__ == "__main__":
    main()

開始查詢NewTaipei的資料...
已更新NewTaipei的資料
開始查詢Taipei的資料...
已更新Taipei的資料
開始查詢Taichung的資料...
已更新Taichung的資料
開始查詢Tainan的資料...
已更新Tainan的資料
開始查詢Kaohsiung的資料...
已更新Kaohsiung的資料
開始查詢Taoyuan的資料...
已更新Taoyuan的資料
開始查詢Yilan的資料...
已更新Yilan的資料
開始查詢Hsinchu的資料...
已更新Hsinchu的資料
開始查詢Miaoli的資料...
已更新Miaoli的資料
開始查詢Changhua的資料...
已更新Changhua的資料
開始查詢Nantou的資料...
已更新Nantou的資料
開始查詢Yunlin的資料...
已更新Yunlin的資料
開始查詢Chiayi的資料...
已更新Chiayi的資料
開始查詢Pingtung的資料...
已更新Pingtung的資料
開始查詢Taitung的資料...
已更新Taitung的資料
開始查詢Hualien的資料...
已更新Hualien的資料
開始查詢Penghu的資料...
已更新Penghu的資料
開始查詢Keelung的資料...
已更新Keelung的資料
開始查詢HsinchuCity的資料...
已更新HsinchuCity的資料
開始查詢ChiayiCity的資料...
已更新ChiayiCity的資料
開始查詢Kinmen的資料...
已更新Kinmen的資料
開始查詢Lianjiang的資料...
已更新Lianjiang的資料
